In [3]:
#widen display #thanks Nick!
from IPython.core.display import display, HTML, clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))

# Project Description:
## Compare local property values against the availability and quality of surrounding businesses.

### Team Members:
Amy, Ebony, Jin, Michael, Syed

### Datasets:
Dallas County Appraisals, Google Maps API

### Questions:
   1.    Is there a correlation between housing prices and the commercial businesses that are available in that area? (comparing high housing price area vs low housing price)
   2.    How does the property value compare with the availability of different types of businesses in the area?
          -  Banks
          -  Grocery Stores
          -  Hospitals
          -  Cafes (Starbucks)
   3.     Where in the Dallas area are the highest and lowest property values?

In [4]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import json
import gmaps
import time

# API Keys
from config import api_key

### Gather and clean data from Dallas County Appraisal District (DCAD) which contains the address and value for all properties in Dallas County (~500k records)

In [5]:
# Read in the Dallas County Appraisal District (DCAD) property values file
file = "Resources/dcad_combined.csv"

#create DataFrame using only the columns we need
dcad_df = pd.read_csv(file, usecols=['PROPERTY_ZIPCODE', 'TOT_VAL'])
dcad_df.head(5)

PROPERTY_ZIPCODE  TOT_VAL
0         750513060   103500
1         750513040   145500
2         750502208   168040
3         750502277   200040
4         750617840   151880

In [6]:
#add column for 5 digit zipcode
dcad_df['ZIPCODE'] = dcad_df['PROPERTY_ZIPCODE'].astype(str).str[:5]
dcad_df.head(5)

PROPERTY_ZIPCODE  TOT_VAL ZIPCODE
0         750513060   103500   75051
1         750513040   145500   75051
2         750502208   168040   75050
3         750502277   200040   75050
4         750617840   151880   75061

In [7]:
#Group Property Values by Zipcode
zip_group = dcad_df.groupby('ZIPCODE')['TOT_VAL']
zip_group.mean().head().map("${:,.2f}".format)

ZIPCODE
75001    $315,208.49
75006    $202,706.61
75007    $200,411.36
75019    $415,797.68
75038    $358,436.94
Name: TOT_VAL, dtype: object

### Create Dataframe from groupby with proprty count mean value, rename columns, and sort

In [8]:
zip_group_df = dcad_df.groupby('ZIPCODE')['TOT_VAL'].agg(('count', 'mean'))\
                    .rename(columns={'count':'PropertyValueCount', 'mean':'MeanPropertyValue'})\
                    .sort_values(by='MeanPropertyValue', ascending=False)\
                    .reset_index()

### Add columns to DataFrame to store the business data we are going to merge

In [9]:
zip_group_df["MeanPropertyDollarValue"] = zip_group_df["MeanPropertyValue"].map("${:,.0f}".format)
zip_group_df['Lat'] = "" 
zip_group_df["Lng"] = ""
zip_group_df["City"] = ""
zip_group_df["State"] = ""
zip_group_df = zip_group_df.rename(columns={"ZIPCODE": "Zipcode"})

In [10]:
zip_group_df.head()

Zipcode  PropertyValueCount  MeanPropertyValue MeanPropertyDollarValue Lat  \
0   75205                6415       1.606362e+06              $1,606,362       
1   75225                7633       1.207824e+06              $1,207,824       
2   75201                1326       9.667580e+05                $966,758       
3   75209                5424       7.230552e+05                $723,055       
4   75230                9337       7.046378e+05                $704,638       

  Lng City State  
0                 
1                 
2                 
3                 
4

## Gather data from Google Maps API about businesses by Zipcode
1. Geocode each Zipcode from DCAD
2. Use lat/lng to search for businesses in each Zipcode
3. Merge with property data which will be used to create charts

### Geocding: loop through each zipcode and update the lat/lng in our dataframe

In [11]:
# create a params dict that will be updated with new zipcode each iteration
params = {"key": api_key}

# Loop through the zipcode pd's and run a lat/long search for each
for index, row in zip_group_df.iterrows():
    base_geocode_url = "https://maps.googleapis.com/maps/api/geocode/json"

    zipcode = row['Zipcode']

    # update address key value to zipcode
    params['address'] = zipcode

    # make request
    zips_lat_lng = requests.get(base_geocode_url, params=params)
    
    # convert to json
    zips_lat_lng = zips_lat_lng.json()
    #print(json.dumps(zips_lat_lng, indent=4, sort_keys=True))
    
    #loop through address to find city
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'locality':
            zip_group_df.loc[index, "City"] = i['long_name']
    
    #loop though address to find state
    for i in zips_lat_lng['results'][0]['address_components']:
        if i['types'][0] == 'administrative_area_level_1':
            zip_group_df.loc[index, "State"] = i['short_name']
    
    #Update Lat, Long in Dataframe
    zip_group_df.loc[index, "Lat"] = zips_lat_lng["results"][0]["geometry"]["location"]["lat"]
    zip_group_df.loc[index, "Lng"] = zips_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [12]:
#visualize to lat/lng have been populated
zip_group_df.head()

Zipcode  PropertyValueCount  MeanPropertyValue MeanPropertyDollarValue  \
0   75205                6415       1.606362e+06              $1,606,362   
1   75225                7633       1.207824e+06              $1,207,824   
2   75201                1326       9.667580e+05                $966,758   
3   75209                5424       7.230552e+05                $723,055   
4   75230                9337       7.046378e+05                $704,638   

       Lat      Lng    City State  
0  32.8326 -96.7976  Dallas    TX  
1  32.8695 -96.7896  Dallas    TX  
2  32.7863 -96.7963  Dallas    TX  
3  32.8539  -96.819  Dallas    TX  
4  32.9005 -96.7869  Dallas    TX

### Function for calling Google Places search and paging through the results

In [13]:
def findPlaces(params):
    #set global variable so name is returned
    global next_page_token
    global results_list

    #Google Places Search
    base_places_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_places_url, params=params)
    
    search_results = json.loads(response.text)
    results_list = search_results["results"]
    
    #Set next page token. Default is None.
    next_page_token = search_results.get("next_page_token", None)
    
    time.sleep(4) #add delay because it can take a moment before the the page token is actually available to make the subsequent call

    return next_page_token, results_list

### Define the types of businesses to lookup:  we chose to focus on essential businesses like banks, supermarkets, hospitals and starbucks

In [14]:
#Google Places Types: https://developers.google.com/places/web-service/supported_types

#The types list can include multiple categories. A count and mean rating will be added to the final DataFrame for each type
types = ['bank','supermarket','hospital', 'cafe']

bank_names = ['bank', 'credit union']
not_supermarket_names = ['dollar', 'cents', 'liquor', 'convenience']
hospital_names = ['hospital', 'medical center', 'parkland']
cafe_names = ['starbucks']

### Count and summarize ratings for each business type using the lat/lng by radius using Google Places API and update our dataframe

In [2]:
# params dictionary to update each iteration
for each_type in types:
    
    #clear console before starting a new type
    clear_output()
    
    params = {
        #3 mi radius. A Zipcode is not returned in the results, so we cannot 
        #match against our zipcode without doing a reverse lookup for every result
        "radius": 4828,
        "types": each_type,
        "key": api_key,
    }
    
    if each_type == 'cafe':
        params["keyword"] = 'starbucks' 
    else: 
        params["keyword"] = None
    
    #variables for the specific column names for the business type we are searching
    count_column = f"{each_type}_count"
    rating_column = f"{each_type}_rating"
    
    #add columns for the business type we are looking up
    zip_group_df[count_column] = 0
    zip_group_df[rating_column] = 0

   
    # Use the lat/lng we recovered to search for businesses
    for index, row in zip_group_df.iterrows():

        rating_sum = 0
        rating_count = 0
        business_count = 0

        next_page_token = "" #initialize the page token

        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]

        # change location each iteration while leaving original params in place
        #params["location"] = f"32.8326,-96.7976" #testing 1 lat, lng
        params["location"] = f"{lat},{lng}"

        while next_page_token != None:
            
            params["pagetoken"] = next_page_token
            
            #call places function
            findPlaces(params)

            #Loop through results list to count each business and get rating for each 
            for each_result in results_list:
                
                #get name in lower case
                business_name_lower = each_result["name"].lower()

                #check to make sure a Rating since not all business have a rating
                if each_result.get("rating") != None:
                    
                    #check for names and keywords based on business type to exclude businesses that are not a good match
                    if each_type == "bank" and any(n in business_name_lower for n in bank_names):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating                        
                    
                    if each_type == "hospital" and any(n in business_name_lower for n in hospital_names):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating     
                    
                    if each_type == "supermarket" and not(any(n in business_name_lower for n in not_supermarket_names)):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating
                        
                    if each_type == "cafe" and any(n in business_name_lower for n in cafe_names):
                        business_count+=1
                        rating = each_result["rating"]
                        rating_sum += rating                             
                                
                    #for printing to console
                    info = f'{row["Zipcode"]} | {each_result["name"]} | location: ({round(each_result["geometry"]["location"]["lat"],2)}, {round(each_result["geometry"]["location"]["lng"],2)}) | rating: {each_result.get("rating",0)}'
                    print(info)
            
            #Set Mean Rating to None if no businesses found
            try:
                mean_rating = rating_sum / business_count
                zip_group_df.loc[index, rating_column] = mean_rating
            except(ZeroDivisionError):
                zip_group_df.loc[index, rating_column] = None
                
            #Set business count in dataframe
            zip_group_df.loc[index, count_column] = business_count
            
        print(f'\nTotal number of {each_type}s found for {row["Zipcode"]}: {business_count} | rating_sum: {round(rating_sum,2)} | avg_rating: {round(mean_rating,2)}')
        print(f'------------------------------------------------------------------------------------------------------\n')

NameError: name 'types' is not defined

In [ ]:
#create file to reference in visualizations (2_visualize-property-values-and-businesses.ipynb) so we don't have to make api calls during presentation
zip_group_df.to_csv('Resources/_zip_group_csv_beta.csv')

In [ ]:
zip_group_df.head()